<a href="https://www.kaggle.com/code/mikedelong/topics-with-bertopic?scriptVersionId=147592261" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
%env TOKENIZERS_PARALLELISM=false
!pip install bertopic

env: TOKENIZERS_PARALLELISM=false
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 64.0 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | / - done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=819467 sha256=eda073c50c14c8fc645c74d15ebf28b5d76e72d8a599c7e469c206b633fa54c7
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=034d871ba5e8032559b23ea41990dba81fcda848faaa111b4d09234445df061c
  Stored in directory: /root/.cache/pip/wheels/6

In [2]:
import pandas as pd
USECOLS = ['description', 'category', 'family', 'mechanic', 'game_id']
df = pd.read_csv(filepath_or_buffer='/kaggle/input/board-games/board_games.csv', index_col=['game_id'], usecols=USECOLS)
df['text'] = df.apply(axis=1, func=lambda x: ' '.join(x.dropna().astype(str)),)
# let's drop everything that isn't alphanumeric
df['text'] = df['text'].apply(func=lambda x: ' '.join([item for item  in x.replace('.', ' ').split() if item.isalpha()]))
df.head()

,description,category,family,mechanic,text
game_id,,,,,
1,Die Macher is a game about seven sequential po...,"Economic,Negotiation,Political","Country: Germany,Valley Games Classic Line","Area Control / Area Influence,Auction/Bidding,...",Die Macher is a game about seven sequential po...
2,Dragonmaster is a trick-taking card game based...,"Card Game,Fantasy",Animals: Dragons,Trick-taking,Dragonmaster is a card game based on an older ...
3,"Part of the Knizia tile-laying trilogy, Samura...","Abstract Strategy,Medieval","Asian Theme,Country: Japan,Knizia tile-laying ...","Area Control / Area Influence,Hand Management,...",Part of the Knizia Samurai is set in medieval ...
4,When you see the triangular box and the luxuri...,Ancient,"Country: Egypt,Promotional Board Games","Action Point Allowance System,Area Control / A...",When you see the triangular box and the large ...
5,"In Acquire, each player strategically invests ...",Economic,3M Bookshelf Series,"Hand Management,Stock Holding,Tile Placement",In each player strategically invests in trying...


In [3]:
from arrow import now
from bertopic import BERTopic
from plotly.express import histogram
from sklearn.feature_extraction.text import CountVectorizer

MAX_DF = 0.2
SAMPLE_FRACTION = 0.5
time_start = now()
large_df = df[['text']].sample(frac=SAMPLE_FRACTION, random_state=2023)
large_model = BERTopic(verbose=True, top_n_words=20, nr_topics=None, language='english', 
                       vectorizer_model=CountVectorizer(stop_words='english', lowercase=True,
                                                       min_df = 3, max_df = MAX_DF),
                      embedding_model='all-MiniLM-L6-v2', )
large_topics, large_probs = large_model.fit_transform(large_df['text'].fillna('').values)
print('After {} our topic says we have {} topics.'.format(now() - time_start, len(set(large_topics))))
print('And we have {} documents with no topic ({}%).'.format(large_topics.count(-1), round(100 * large_topics.count(-1)/len(large_topics))))
histogram(x=[item for item in large_topics if item > -1], title = 'topic > -1 histogram')

/opt/conda/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/conda/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/conda/lib/python3.10/site-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopython' keyword argume

Batches:   0%|          | 0/165 [00:00<?, ?it/s]

2023-10-22 16:23:04,227 - BERTopic - Transformed documents to Embeddings
2023-10-22 16:23:45,519 - BERTopic - Reduced dimensionality
2023-10-22 16:23:45,777 - BERTopic - Clustered reduced embeddings


After 0:06:03.937839 our topic says we have 77 topics.
And we have 1885 documents with no topic (36%).


In [4]:
large_model.visualize_topics(width=1000, height=1000)

Once we use the count vectorizer to limit very common tokens we end up with topics that for the most part describe game stories and subject matter rather than game mechanics.

What's going on in the documents where we have no topic?

In [5]:
large_df['topic'] = large_topics
small_df = large_df[large_df['topic'] == -1].drop(columns=['topic'])
small_df.shape

(1885, 1)

In [6]:
time_start = now()
small_model = BERTopic(verbose=True, top_n_words=20, nr_topics=None, language='english',
                       vectorizer_model=CountVectorizer(stop_words='english', lowercase=True),
                       embedding_model='all-MiniLM-L6-v2', )
small_topics, small_probs = small_model.fit_transform(small_df['text'].fillna('').values)
print('After {} our topic says we have {} topics.'.format(now() - time_start, len(set(small_topics))))
print('And we have {} documents with no topic ({}%).'.format(small_topics.count(-1), round(100 * small_topics.count(-1)/len(small_topics))))
histogram(x=[item for item in small_topics if item > -1], title = 'topic > -1 histogram')

Batches:   0%|          | 0/59 [00:00<?, ?it/s]

2023-10-22 16:25:55,494 - BERTopic - Transformed documents to Embeddings
2023-10-22 16:26:05,632 - BERTopic - Reduced dimensionality
2023-10-22 16:26:05,730 - BERTopic - Clustered reduced embeddings


After 0:02:09.022538 our topic says we have 20 topics.
And we have 1054 documents with no topic (56%).


In [7]:
small_model.visualize_topics()

In [8]:
small_model.get_topic(topic=-1)

[('game', 0.031236879290575864),
 ('card', 0.027346121710021733),
 ('cards', 0.026945045739168155),
 ('player', 0.025694407520712677),
 ('players', 0.0254647221327025),
 ('play', 0.016425224946815854),
 ('points', 0.013988894761001671),
 ('new', 0.013636505383225543),
 ('hand', 0.012168544863746654),
 ('board', 0.012051767123678178),
 ('action', 0.011690191436667628),
 ('deck', 0.011021098721260537),
 ('games', 0.01065302443990913),
 ('turn', 0.01058613471700939),
 ('dice', 0.010550320531686775),
 ('different', 0.010053690809886943),
 ('set', 0.009967733458207598),
 ('number', 0.009315355552375637),
 ('use', 0.009202293688388305),
 ('end', 0.008941143382516838)]

In [9]:
small_model.get_topic(topic=0)

[('war', 0.048004441368025424),
 ('units', 0.030160832549044014),
 ('battle', 0.029417971328935186),
 ('game', 0.026425161295271254),
 ('ii', 0.026109779415076542),
 ('combat', 0.022482034435271595),
 ('rules', 0.020819299695419288),
 ('campaign', 0.018727492696705695),
 ('german', 0.01869612719318647),
 ('forces', 0.0158151271039665),
 ('battles', 0.01567451445120348),
 ('series', 0.015250720013132548),
 ('world', 0.014908838585981887),
 ('british', 0.014832171959571292),
 ('unit', 0.014667249247952145),
 ('allied', 0.01407390209371897),
 ('army', 0.0140219115264202),
 ('air', 0.013657727373363123),
 ('counters', 0.013638070942985215),
 ('map', 0.01359301162725572)]

For the most part these topics just contain documents where we don't know much apart from game mechanics.